In [16]:
import os

old_folder = r'E:\Data\20240606-4T1noMMt3v210xPE2_colony0529Live3_MF8'

before_position_file = os.path.join(old_folder, '10x_positions_before.txt')

new_folder = r'E:\Data\20240617-4T1noMMt3v210xPE2_colony0529Live3_sample0606_intBCv2Editv3'

after_position_file = os.path.join(new_folder, '10x_positions_after.txt')

print(os.path.exists(before_position_file), os.path.exists(after_position_file))

True True


# Calculate rotation matrix

In [17]:
import numpy as np
import os
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [18]:
before_position_file, after_position_file

('E:\\Data\\20240606-4T1noMMt3v210xPE2_colony0529Live3_MF8\\10x_positions_before.txt',
 'E:\\Data\\20240617-4T1noMMt3v210xPE2_colony0529Live3_sample0606_intBCv2Editv3\\10x_positions_after.txt')

In [19]:
R, T = align_manual_points(before_position_file, after_position_file, save=True, save_folder=new_folder)
R, T

(array([[ 0.99997513, -0.0070522 ],
        [ 0.0070522 ,  0.99997513]]),
 array([ 347.66504317, -408.15777806]))

In [20]:
# Translate 60x positions
old_fov_position_file = os.path.join(old_folder, r'positions_all.txt')
print(old_fov_position_file)
old_fov_positions = np.loadtxt(old_fov_position_file, delimiter=',')
print(len(old_fov_positions))

E:\Data\20240606-4T1noMMt3v210xPE2_colony0529Live3_MF8\positions_all.txt
333


In [21]:
new_fov_positions = np.dot(old_fov_positions, R) + T
#print(new_fov_positions)
new_fov_position_savefile = os.path.join(new_folder, r'positions_all.txt')
# save
print(f"Save {len(new_fov_positions)} coordinates into the file: {new_fov_position_savefile}")
np.savetxt(new_fov_position_savefile, np.round(new_fov_positions, 2), delimiter=',', fmt='%.2f')

Save 333 coordinates into the file: E:\Data\20240617-4T1noMMt3v210xPE2_colony0529Live3_sample0606_intBCv2Editv3\positions_all.txt
